In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import copy

In [44]:
def phase_dependent_response(driver_values, t_dev, responses, thresholds):
    #Thresholds are the thresholds in development time where the different growth phases change
    #Responses are the response functions, index starting at 'before the first threshold'
    #driver values are the inputs to the response function
    #t_dev is the (cts) development time
    phase = np.digitize(t_dev, thresholds)
    response = np.zeros(driver_values.shape)
    for phase_index in range(len(responses)):
        response += (phase == phase_index)*responses[phase_index](driver_values)
    return response

In [43]:
def dev_under_response(response, ds_driver, thresholds, maturity_t_dev, emergence_date):
    # Response is the rate response to driver values. Driver values are the input to this response. Maturity_t_dev is the t_dev value where we should stop running.
    t_dev = np.zeros(ds_driver.isel(time=0).values.shape) #Continuous development time. When this passes through some thresholds then have change in phase.
    dev_time_series = [t_dev.copy()]
    i_day = emergence_date
    while np.any(t_dev < maturity_t_dev):
        i_day +=1
        driver_values = ds_driver.isel(time=i_day).values
        t_dev += response(driver_values, t_dev) #Rate of change of development stage
        dev_time_series.append(t_dev.copy())
    return np.stack(dev_time_series)

In [59]:
def get_phase_dates(dev_time_series, thresholds):
    phase_dates_array = np.zeros((len(thresholds), dev_time_series.shape[1], dev_time_series.shape[2]))
    for i_x in range(dev_time_series.shape[1]):
        for i_y in range(dev_time_series.shape[2]):
            phase_dates_array[:, i_x, i_y] = np.digitize(thresholds, dev_time_series[:, i_x, i_y]) #Note that the thresholds are NOT the bins for numpy digitize!
    return phase_dates_array

Need:
 - More realistic temperature response functions
 - What to do about emergence date
 - Relate to phen data?

In [45]:
ts = dev_under_response(lambda x, y: phase_dependent_response(x, y, resps, thresholds), ds_mean['tas'], [3, 6, 10], 10, 15000)

In [50]:
ts[:, 0, 0]

array([ 0.        ,  1.        ,  2.        ,  3.        ,  4.        ,
        5.        ,  6.        ,  7.        ,  7.56      ,  8.25999999,
       11.05999994, 11.05999994, 11.05999994])

In [58]:
print(type(ts.shape[1]))

<class 'int'>


In [60]:
pds = get_phase_dates(ts, [3, 6, 10])

In [64]:
pds[:, 10, 11]

array([3., 3., 8.])

In [ ]:
dev_under_response(lambda x, y: phase_dependent_response(x, y, resps, thresholds), ds_mean['tas'], [3, 6, 10], 10, 15000)#np.datetime64('2000-03-25')
#[lambda x: 2*x, lambda x: 0.5*x, lambda x: 0.1*x, lambda x: 0*x]

In [27]:
ds_mean = xr.open_dataset('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\tas_hyras_5_1951_2020_v5-0_de.nc')
ds_mean = ds_mean.where((ds_mean['x'] > 4020000)*(ds_mean['x'] < 4080000)*(ds_mean['y'] > 2520000)*(ds_mean['y'] > 2580000), drop = True)

In [41]:
resps = [lambda x: 2*x*(x > 0) + 1*(x<= 0), lambda x: 0.5*x*(x > 0) + 1*(x<= 0), lambda x: 0.7*x*(x > 0) + 1*(x<= 0), lambda x: 0*x]
Ts = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
thresholds = [3, 6, 10]
#print(phase_dependent_response(Ts, Ts, resps,  thresholds))

In [29]:
ds_mean

<xarray.Dataset>
Dimensions:             (time: 25568, bnds: 2, x: 12, y: 124)
Coordinates:
  * time                (time) datetime64[ns] 1951-01-01T12:00:00 ... 2020-12...
    lon                 (y, x) float64 ...
    lat                 (y, x) float64 ...
  * x                   (x) float32 4.022e+06 4.028e+06 ... 4.072e+06 4.078e+06
  * y                   (y) float32 2.582e+06 2.588e+06 ... 3.192e+06 3.198e+06
Dimensions without coordinates: bnds
Data variables:
    time_bnds           (time, bnds, x, y) datetime64[ns] 1951-01-01 ... 2021...
    x_bnds              (x, bnds, y) float32 4.02e+06 4.02e+06 ... 4.08e+06
    y_bnds              (y, bnds, x) float32 2.58e+06 2.58e+06 ... 3.2e+06
    crs_HYRAS           (x, y) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    tas                 (time, y, x) float32 -0.8 -0.8 -0.8 -0.7 ... nan nan nan
    number_of_stations  (time, x, y) float64 817.0 817.0 ... 1.456e+03 1.456e+03
Attributes: (12/19)
    source:                 surface observations
    Conventions:            CF-1.8
    realization:            v5.0
    project_id:             HYRAS
    input_data_status:      non-realtime
    level_type:             surface
    ...                     ...
    horizontal_resolution:  5 km
    title:                  gridded_temperature_dataset_(HYRAS-DE TAS)
    institution:            Deutscher Wetterdienst (DWD)
    creation_date:          created at 2022-12-04 13:12:56
    filename:               tas_hyras_5_1951_2020_v5-0_de.nc
    unique_dataset_id:      DWD_HYRAS_DE_tas_v5.0_1951_2020_day_00638C9CD8